In [1]:
import pandas as pd
import os
import random
import time
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
data_dir = '/opt/ml/input/data/'
csv_file_path = os.path.join(data_dir, 'train_data_tag_fe.csv')
df = pd.read_csv(csv_file_path)
df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,year,month,day,hour,...,test_type_lvl_cat,test_total_answer,tag_ans_mean,tag_ans_sum,tag_ans_std,tag_time_mean,tag_time_std,tag_lvl,tag_lvl_cat,tag_total_answer
0,0,A060001001,A060000001,1,1585009031,7224,2020,3,24,0,...,1,0,0.955022,637,0.207410,15.587706,32.699735,16.321821,0,0
1,0,A060001002,A060000001,1,1585009034,7225,2020,3,24,0,...,1,1,0.913187,3040,0.281603,29.133073,41.916419,31.902632,0,0
2,0,A060001003,A060000001,1,1585009042,7225,2020,3,24,0,...,1,2,0.913187,3040,0.281603,29.133073,41.916419,31.902632,0,1
3,0,A060001004,A060000001,1,1585009049,7225,2020,3,24,0,...,1,3,0.913187,3040,0.281603,29.133073,41.916419,31.902632,0,2
4,0,A060001005,A060000001,1,1585009056,7225,2020,3,24,0,...,1,4,0.913187,3040,0.281603,29.133073,41.916419,31.902632,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,1591339821,438,2020,6,5,6,...,0,1389,0.689706,2814,0.462671,50.524265,59.503996,73.254797,4,4079
2266582,7441,A040165001,A040000165,1,1597971999,8836,2020,8,21,1,...,1,1092,0.697874,2199,0.459253,43.622342,60.170574,62.507503,3,3147
2266583,7441,A040165002,A040000165,1,1597972010,8836,2020,8,21,1,...,1,1093,0.697874,2199,0.459253,43.622342,60.170574,62.507503,3,3148
2266584,7441,A040165003,A040000165,1,1597972056,8836,2020,8,21,1,...,1,1094,0.697874,2199,0.459253,43.622342,60.170574,62.507503,3,3149


In [3]:
def user_fe(df):
    #유저별 시퀀스를 고려하기 위해 아래와 같이 정렬
    #유저들의 문제 풀이수, 정답 수, 정답률을 시간순으로 누적해서 계산
    df['user_correct_answer'] = df.groupby('userID')['answerCode'].transform(lambda x: x.cumsum().shift(1))
    df['user_total_answer'] = df.groupby('userID')['answerCode'].cumcount()
    df['user_acc'] = df['user_correct_answer']/df['user_total_answer']



    ## 임시
    
    ## tag의 평균 정답률, 정답 총합, 표준편차
    correct_k = df.groupby(['KnowledgeTag'])['answerCode'].agg(['mean', 'sum','std'])
    correct_k.columns = ["tag_ans_mean", 'tag_ans_sum','tag_ans_std']
    df = pd.merge(df, correct_k, on=['KnowledgeTag'], how="left")
    
    ## 태그의 평균 풀이 시간, 표준편차
    time_k = df.groupby(['KnowledgeTag'])['time'].agg(['mean', 'std'])
    time_k.columns = ["tag_time_mean", 'tag_time_std']
    df = pd.merge(df, time_k, on=['KnowledgeTag'], how="left")

    ## 태그 난이도
    df['tag_lvl'] = df['tag_time_mean'] / df['tag_ans_mean']
    # 범주화
    tag_cat_num = 10
    df['tag_lvl_cat'] = pd.qcut(df['tag_lvl'],tag_cat_num,labels=[i for i in range(tag_cat_num)])

    ## 태그 노출 횟수
    df['tag_total_answer'] = df.groupby('KnowledgeTag')['answerCode'].cumcount()
    return df

In [4]:
df = user_fe(df)

In [5]:
df
df.to_csv('/opt/ml/input/data/train_data_user_fe.csv',index=False)